In [42]:
import requests
from pprint import pprint
import json
import pandas as pd

API_KEY='5e5cec06054aaa4b312859dd97fb9a94c3211af602e0af17cd8350df3b15dbd1'

In [ ]:


# import requests

# Define the API endpoint to find locations in Kathmandu

url = "https://api.openaq.org/v3/locations?coordinates=27.7172,85.3240&radius=12000"

 
# headers
headers = {'X-API-Key': API_KEY}

# Define query parameters to get sensor id for Kathmandu
# params = {
#     "city": "Kathmandu",
#     "country": "NP",
#     "limit": 1
# }

# # Fetch data
response = requests.get(url,  headers=headers)
if response.status_code == 200:
    data = response.json()
    for location in data["results"]:
        pprint(location)
        
else:
    print(f"Failed to fetch data: {response}")


# Define the API endpoint to get data for a specific sensor
# List of sensor IDs for O₃
sensor_ids = [7713, 7712]  # Embassy Kathmandu and US Diplomatic Post: Phora Durbar Kathmandu

# # Fetch data for each sensor
# for sensor_id in sensor_ids:
#     url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements/daily?date_from=2023-01-01&date_to=2023-01-31&limit=1000"

#     response = requests.get(url, headers=headers)
#     if response.status_code == 200:
#         data = response.json()
#         with open(f"sensor_{sensor_id}_daily_o3.json", "w") as file:
#             json.dump(data["results"], file, indent=4)
#         print(f"Data saved for sensor {sensor_id}")
#     else:
#         print(f"Failed to fetch data for sensor {sensor_id}: {response.status_code}")
        
# import requests



In [210]:


# Define the sensor ID for O₃ at US Diplomatic Post: Phora Durbar Kathmandu
sensor_id = 7712  # Replace with the correct sensor ID for O₃

# Define query parameters
params = {
    "datetime_from": "2024-01-01T00:00:00Z",  # Start date in ISO 8601 format
    "datetime_to": "2025-02-24T23:59:59Z",  # End date in ISO 8601 format
    "limit": 1000,  # Number of results per page
    "page": 1  # Start with page 1
}

# Define the API endpoint for daily measurements
url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements/daily"


# Fetch all pages of data
all_results = []  # Store all results here
while True:
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        all_results.extend(data["results"])  # Add results to the list

        # Check if there are more pages
        if data["meta"]["found"] <= params["page"] * params["limit"]:
            break  # Exit the loop if all data has been fetched

        params["page"] += 1  # Move to the next page
    else:
        print(f"Failed to fetch data: {response.status_code}")
        print(f"Response: {response.text}")  # Print the error response for debugging
        break

print(f"Total daily measurements fetched: {len(all_results)}")

Total daily measurements fetched: 422


In [211]:

# extract the dates
dates = [result["period"]["datetimeFrom"]['utc'] for result in all_results]


len(dates)

dates = pd.to_datetime(dates).date # convert to datetime object and extract date

dates.min(), dates.max()



(datetime.date(2023, 12, 31), datetime.date(2025, 2, 24))

In [212]:


o3_values = [result["summary"]["median"] for result in all_results] # extract the median values

In [213]:
len(o3_values) # check the length of the values



422

In [214]:
# create a dataframe containing the dates and O₃ values
df = pd.DataFrame({"Date": dates[1:], "o3": o3_values[1:]})
df['o3'] = df['o3'].apply(lambda x: f"{x:.4f}")
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.shape

(421, 2)

In [215]:
dataset = pd.read_csv("../year_2024/Kathmandu_2024.csv")
# dataset['o3'] = df['o3'].apply(lambda x: f"{x:.4f}")
dataset['Date'] = pd.to_datetime(dataset['Date']).dt.date

# dropping the last row
dataset = dataset.iloc[:,:-1]




dataset.shape

(421, 10)

In [216]:
# Merge the two datasets on the date column
merged_df= pd.merge(dataset,df, on="Date", how="left")

In [217]:
merged_df.sample(5)

,Date,dew,humidity,pm1,pm10,pm25,pressure,temperature,wind-gust,wind-speed,o3
144,2024-05-24,18.5,75.5,79.0,62.0,109.0,1013.0,23.5,NaN,1.1,-0.9990
410,2025-02-14,7.5,78.5,166.0,98.0,174.0,1013.0,11.0,NaN,1.5,-0.9990
190,2024-07-09,22.0,89.5,NaN,35.0,85.0,1012.0,23.9,NaN,1.1,-0.9990
409,2025-02-13,5.0,73.5,151.0,92.0,166.0,1014.5,12.0,NaN,1.5,-0.9990
191,2024-07-10,22.0,95.2,NaN,25.0,70.0,1013.0,23.0,NaN,1.2,-0.5010


In [ ]:
# convert the merged dataset to a csv file
merged_df.to_csv("../year_2024/Kathmandu_2024.csv", index=False)